In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.animation as animation
import time
import struct
import tensorflow as tf
import random as rd

from array import array
# import keras._tf_keras.keras as keras 
# from keras._tf_keras.keras
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression

# my project
from module.conf import PROJECT_DIR

# matplotlib.use("QTAgg")
%matplotlib inline

1. Load data:
- Train data: 60k 28x28 images
- Test data: 

In [2]:
mnist_path = "/data/sample/mnist"
training_images_filepath = "".join([PROJECT_DIR, mnist_path, "/train-images.idx3-ubyte"])
training_labels_filepath = "".join([PROJECT_DIR, mnist_path, "/train-labels.idx1-ubyte"])
test_images_filepath = "".join([PROJECT_DIR, mnist_path, "/t10k-images.idx3-ubyte"])
test_labels_filepath = "".join([PROJECT_DIR, mnist_path, "/t10k-labels.idx1-ubyte"])

def read_images_labels(images_filepath, labels_filepath) -> tuple:
    labels = []
    with open(labels_filepath, 'rb') as file:
        magic, size = struct.unpack(">II", file.read(8))
        if magic != 2049:
            raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
        # labels = array("B", file.read())
        labels = array("B", file.read())

    with open(images_filepath, 'rb') as file:
        magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
        if magic != 2051:
            raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
        image_data = array("B", file.read())       
     
    images = []
    # for i in range(size):
    #     images.append([0] * rows * cols)
    for i in range(size):
        img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
        img = img.reshape(28, 28)
        # images[i][:] = img
        images.append(img)
    
    return images, labels

def load_data() -> tuple:
    x_train, y_train = read_images_labels(training_images_filepath, training_labels_filepath)
    x_test, y_test = read_images_labels(test_images_filepath, test_labels_filepath)
    return (x_train, y_train),(x_test, y_test)

(X_train, y_train), (X_test, y_test) = load_data()

In [ ]:
# print(f"{type(X_train[0])}")
# mnist = tf.keras.datasets.mnist

# (x_train, y_train), (x_test, y_test) = mnist.load_data()


In [3]:
X_train = np.asarray(X_train)/255
y_train = np.asarray(y_train)
X_test  = np.asarray(X_test)/255
y_test  = np.asarray(y_test)

## 1. Activation functions:

### 1.1. Linear:
$ \begin{align}
f(\mathbf z) &= \mathbf x \\
\rightarrow \frac{\partial f(\mathbf z)}{\partial \mathbf z} &=\mathbf 1 \\ 
\end{align} $

### 1.2. ReLU:
$\begin{align}
ReLU(\mathbf z) &= \max(\mathbf z, \mathbf 0) \\
\rightarrow \frac{\partial ReLU(\mathbf z)}{\partial \mathbf z} &= \begin{cases}
x_i = 1 \text{ if } x_i > 0 \\
x_i = 0 \text{ if } x_i \leqslant 0 \\
\end{cases} \\
\end{align}$

### 1.2. Sigmoid:
$\begin{align}
\sigma(\mathbf z) &= \frac{1}{1 + e^{-z}} \\
\rightarrow \frac{\partial\sigma(\mathbf z)}{\partial \mathbf z} &= \sigma(\mathbf z)\cdot\left(1 - \sigma(\mathbf z)\right) \\ 
\end{align} $

### 1.3. Softmax:
$\begin{align}
\sigma(\mathbf z) &= \frac{e^{\mathbf z}}{\sum_{i=1}^{C}e^{z_i}} \\
\rightarrow \frac{\partial \sigma(\mathbf z)}{\partial \mathbf z} &= \sigma(z_i) \cdot (\delta_{ij} - \sigma(z_j)) 
\rightarrow \delta_{ij} = \begin{cases} 
1 \text{ if } i = j \\
0 \text{ if } i \neq j  
\end{cases} \\
&= diag(\mathbf z) - \mathbf z * \mathbf z^T \\
C &\text{ is number of class} \\
diag &\text{ is diagonal matrix }
\end{align}$

## 2. Loss function:

### 2.1. Cross Entropy:
$\begin{align}
CrossEntropy = - \log(\hat{y}_{true})\\
\end{align}$

### 2.2. Categorical Crossentropy:
$\begin{align}
Y &\text{ is label in one-hot matrix} \\
\hat{Y} &\text{ is predicted matrix} \\
C &\text{ is number of classes}\\
L &= -\sum_{i=1}^{C} Y_i \log(\hat{Y}_{i}) \\
\end{align}$

### 2.3. Sparse Categorical Crossentropy:
$\begin{align}
\hat{Y} &= A_n = softmax(Z) \\
Z &\text{ is } n \times C \text{ matrix. n is number of samples, C is number of classes} \\
CrossEntropy_i &= -\log(\hat{y}_{i, y_{sparse}}) \\
CrossEntropy &\text{ is a vector size n} \\
\rightarrow \frac{\partial L}{\partial Z_{i,j}} &= \hat{Y}_{i,j} - \delta(j, y_{sparse,i}) 
\rightarrow \delta(j, y_{sparse,i}) = \begin{cases}
1 \text{ if } j = y_{sparse,i} \\
0 \text{ if } j \neq y_{sparse,i}\\
\end{cases} \\
\rightarrow \frac{\partial L}{\partial Z} &= \hat{Y} - SparseLabels \\
SparseLabels &\text{ can be considered as one-hot matrix}
\end{align}$

In [ ]:
# model = tf.keras.models.Sequential(layers=[
#     tf.keras.layers.Flatten(input_shape=(28, 28,)),
#     tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu),
#     tf.keras.layers.Dense(units=128, activation=tf.keras.activations.sigmoid),
#     # tf.keras.layers.Dropout(rate=0.2),
#     tf.keras.layers.Dense(units=10, activation=tf.keras.activations.softmax)
# ])

In [ ]:
# predictions = model(X_train[0]).numpy()
# predictions
# tf.nn.softmax(predictions).numpy()
# model.summary()

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# loss_fn = tf.keras.losses.CategoricalHinge()
# loss_fn = tf.keras.losses.MeanSquaredLogarithmicError()
# model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#               loss=tf.keras.losses.BinaryCrossentropy(),
#               metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.FalseNegatives()])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=loss_fn,
              metrics=["accuracy"])

In [ ]:
# X_train, X_test = np.asarray(X_train) / 255.0, np.asarray(X_test) / 255.0
# print(X_test)
model.fit(x=X_train, y=y_train, epochs=50, batch_size=600, workers=8, use_multiprocessing=False)

In [ ]:
model.evaluate(X_test,  y_test, verbose=2)
c = 0
cp = 0
for i in range(100):
    test_indx = rd.randint(0, len(y_test)-1)
    x_test_ = np.asarray([X_test[test_indx]])

    # test_indx = rd.randint(0, len(y_train)-1)
    # x_test_ = np.asarray([X_train[test_indx]])

    result = model.predict(x=x_test_, verbose=0)
    # result = tf.nn.softmax(result).numpy()
    y_test_ = y_test
    # if result.max() >= 0.5:
    if result.argmax() != y_test_[test_indx]:
        c+=1
        print(f"- [{i}]:img[{test_indx}]:{result}\npred:{result.max()}\npredict:{result.argmax()} solve:{y_test_[test_indx]}")
    else:
        print(f"+ [{i}]:img[{test_indx}]:{result}\npred:{result.max()}\npredict:{result.argmax()} solve:{y_test_[test_indx]}")
    # else:
    #     print(f"= [{i}]:img[{test_indx}]:{result}\npred:{result.max()}\npredict:{result.argmax()} solve:{y_test_[test_indx]}")
    #     cp+=1
print(f"error: {c} can not pred:{cp}")

In [ ]:
def show_image(img_data: np.ndarray) -> tuple:
    fig, axes = plt.subplots(figsize=(1.60, 1.20))
    axes.imshow(X=img_data, cmap="gray")
    return fig, axes

# print(y_test[5854])
show_image(X_test[4823])
plt.show()

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))

Trong quá trình **backpropagation**, khi sử dụng hàm softmax trong lớp đầu ra của một mạng nơ-ron, ta cần tính toán gradient của hàm mất mát (thường là categorical cross-entropy) đối với các trọng số. Điều này yêu cầu tính toán đạo hàm của hàm softmax.

### 1. **Hàm Softmax và Hàm Mất Mát Cross-Entropy**

Giả sử đầu ra của mạng nơ-ron là một vector \( \mathbf{z} = [z_1, z_2, \dots, z_n] \). Hàm softmax được định nghĩa như sau:

\[
\sigma(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}
\]

Giả sử nhãn đúng là \( y \), hàm mất mát cross-entropy có dạng:

\[
L = -\sum_{i=1}^{n} y_i \log(\sigma(z_i))
\]

### 2. **Đạo Hàm Của Hàm Mất Mát Đối Với Đầu Ra Của Softmax**

Để thực hiện backpropagation, ta cần tính đạo hàm của hàm mất mát \( L \) đối với mỗi đầu ra của softmax \( z_i \):

\[
\frac{\partial L}{\partial z_i}
\]

Sử dụng quy tắc dây chuyền, ta có:

\[
\frac{\partial L}{\partial z_i} = \sum_{j=1}^{n} \frac{\partial L}{\partial \sigma(z_j)} \cdot \frac{\partial \sigma(z_j)}{\partial z_i}
\]

#### 2.1 **Đạo Hàm Của Hàm Mất Mát Với Softmax**

Đạo hàm của hàm mất mát \( L \) đối với đầu ra của softmax \( \sigma(z_j) \) là:

\[
\frac{\partial L}{\partial \sigma(z_j)} = \sigma(z_j) - y_j
\]

#### 2.2 **Đạo Hàm Của Softmax**

Đạo hàm của hàm softmax được tính như sau:

\[
\frac{\partial \sigma(z_j)}{\partial z_i} = 
\begin{cases} 
\sigma(z_i) \cdot (1 - \sigma(z_i)) & \text{nếu } i = j \\
-\sigma(z_i) \cdot \sigma(z_j) & \text{nếu } i \neq j 
\end{cases}
\]

### 3. **Vectorization (Tính Toán Dạng Vector)**

Trong thực hành, tính toán đạo hàm của hàm softmax được thực hiện thông qua vectorization để tối ưu hóa hiệu suất. Giả sử \( \mathbf{\sigma} \) là vector chứa các giá trị softmax \( \sigma(z_1), \sigma(z_2), \dots, \sigma(z_n) \), ta có:

\[
\frac{\partial \mathbf{\sigma}}{\partial \mathbf{z}} = \text{Jacobian}(\mathbf{\sigma}) = \mathbf{S} - \mathbf{\sigma} \cdot \mathbf{\sigma}^T
\]

trong đó:

- \( \mathbf{S} \) là ma trận chéo (diagonal matrix) với các phần tử \( \sigma(z_i) \) trên đường chéo chính.
- \( \mathbf{\sigma} \cdot \mathbf{\sigma}^T \) là tích ngoài (outer product) của vector \( \mathbf{\sigma} \).

Cụ thể hơn:

\[
\text{Jacobian}(\mathbf{\sigma}) = \text{diag}(\sigma) - \sigma \cdot \sigma^T
\]

### 4. **Gradient Đối Với Vector Đầu Ra**

Cuối cùng, gradient của hàm mất mát đối với vector \( \mathbf{z} \) (đầu ra trước softmax) có thể được biểu diễn dưới dạng vectorized:

\[
\frac{\partial L}{\partial \mathbf{z}} = \mathbf{\sigma} - \mathbf{y}
\]

Đây là dạng vectorized của gradient, rất quan trọng trong quá trình huấn luyện mô hình với backpropagation vì nó cho phép tính toán gradient một cách hiệu quả, đặc biệt là khi làm việc với các tập dữ liệu lớn và các mô hình có nhiều lớp.

### Tóm Lược

- **Đạo hàm của hàm softmax** có thể được biểu diễn dưới dạng ma trận Jacobian.
- Trong quá trình **backpropagation**, gradient của hàm mất mát đối với đầu ra trước softmax \( z_i \) có dạng vectorized: \( \frac{\partial L}{\partial \mathbf{z}} = \mathbf{\sigma} - \mathbf{y} \).
- Vectorization giúp tính toán gradient nhanh chóng và hiệu quả hơn khi huấn luyện các mô hình học sâu.